In [12]:
#Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance

In [13]:
# Read the dataset
df = pd.read_csv("inputs-for-ml/final_ml_data.csv")

In [14]:
# Select features and target
X = df[["slope", "elevation", "north_gps", "east_gps", "vertical_gps", "coherence","los_insar"]]
y = df["bias"]

In [15]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)

In [16]:
# Define regression models
models = {
    "Linear Regression": LinearRegression(),
    "Support Vector Regression": SVR(),
    "Neural Network Regression": MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42),
    "Decision Tree Regression": DecisionTreeRegressor(random_state=42),
    "Random Forest Regression": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting Regression": GradientBoostingRegressor(n_estimators=100, random_state=42)
}

In [ ]:
# Initialize performance list
performance_list = []

# Train and evaluate models
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    performance_list.append({"Model": name, "RMSE": rmse, "R2": r2})

    # Plot predicted vs actual and Q-Q plot
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Scatter plot: Predicted vs Actual
    sns.regplot(
        x=y_test, y=y_pred,
        scatter_kws={"s": 20},
        line_kws={"color": "red", "linestyle": "--"},
        ax=axes[0]
    )
    axes[0].set_xlabel("Actual Bias")
    axes[0].set_ylabel("Predicted Bias")
    axes[0].set_title(f"{name} - Predicted vs Actual")
    axes[0].grid(True)
    axes[0].set_aspect('equal', adjustable='box')

    textstr = f"RMSE: {rmse:.3f}\nR²: {r2:.3f}"
    axes[0].text(
        0.05, 0.95, textstr,
        transform=axes[0].transAxes,
        fontsize=10,
        verticalalignment='top',
        bbox=dict(boxstyle="round", facecolor="white", alpha=0.7)
    )

    # Q-Q plot: Residuals
    residuals = y_test - y_pred
    stats.probplot(residuals, dist="norm", plot=axes[1])
    axes[1].get_lines()[1].set_color('red')
    axes[1].set_xlabel("Actual Bias Quantiles")
    axes[1].set_ylabel("Estimated Bias Quantiles")
    axes[1].set_title(f"{name} - Q-Q Plot of Residuals")
    axes[1].grid(True)

    plt.suptitle(f"{name} - Performance Analysis", fontsize=14, weight='bold')
    plt.tight_layout(rect=[0, 0, 1, 0.95], w_pad=4)
    plt.show()

In [ ]:
# Create performance DataFrame
performance_df = pd.DataFrame(performance_list)

# Clean and format columns
performance_df.rename(columns=lambda x: x.strip().upper(), inplace=True)
performance_df = performance_df.round({"RMSE": 3, "R2": 3})
performance_df = performance_df.sort_values(by="R2", ascending=False).reset_index(drop=True)
performance_df.index += 1
performance_df.index.name = "Rank"

# Print performance table
print(performance_df)

In [19]:
# Extended Hyperparameter Tuning for All Models

# Redefine models with hyperparameter grids
param_grids = {
    "Ridge Regression": {
        "model": RidgeCV(alphas=[0.1, 1.0, 10.0], cv=5),
        "tune": False
    },
    "Support Vector Regression": {
        "model": SVR(),
        "params": {
            "C": [0.1, 1, 10],
            "epsilon": [0.01, 0.1, 0.2],
            "kernel": ["rbf"]
        }
    },
    "Neural Network Regression": {
        "model": MLPRegressor(max_iter=1000, random_state=42),
        "params": {
            "hidden_layer_sizes": [(50,), (100,), (100, 50)],
            "alpha": [0.0001, 0.001],
            "learning_rate_init": [0.001, 0.01]
        }
    },
    "Decision Tree Regression": {
        "model": DecisionTreeRegressor(random_state=42),
        "params": {
            "max_depth": [None, 10, 20],
            "min_samples_split": [2, 5, 10]
        }
    },
    "Random Forest Regression": {
        "model": RandomForestRegressor(random_state=42),
        "params": {
            "n_estimators": [50, 100, 200],
            "max_depth": [None, 10, 20],
            "min_samples_split": [2, 5, 10]
        }
    },
    "Gradient Boosting Regression": {
        "model": GradientBoostingRegressor(random_state=42),
        "params": {
            "n_estimators": [50, 100, 200],
            "learning_rate": [0.01, 0.1],
            "max_depth": [3, 5, 7]
        }
    }
}


In [ ]:

# Initialize dictionaries to store best models and importances
best_models = {}
feature_importance_dict = {}

# Iterate over models and tune
for name, settings in param_grids.items():
    model = settings["model"]
    
    if settings.get("tune", True):
        grid_search = GridSearchCV(model, settings["params"], cv=5, scoring="r2", n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
    else:
        model.fit(X_train, y_train)
        best_model = model

    best_models[name] = best_model

    # Feature Importance
    if hasattr(best_model, "feature_importances_"):
        feature_importance_dict[name] = best_model.feature_importances_
    elif hasattr(best_model, "coef_"):
        feature_importance_dict[name] = np.abs(best_model.coef_)
    else:
        # Use permutation importance for models that don't have built-in importance
        result = permutation_importance(best_model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
        feature_importance_dict[name] = result.importances_mean

In [ ]:
# Collect and Display Feature Importances

# Create feature importance DataFrame
feature_importance_df = pd.DataFrame(feature_importance_dict, index=df[["slope", "elevation", "north_gps", "east_gps", "vertical_gps", "coherence", "los_insar"]].columns)

# Normalize feature importance values for comparison
feature_importance_df = feature_importance_df.div(feature_importance_df.sum(axis=0), axis=1)

# Plot feature importances
fig, ax = plt.subplots(figsize=(12, 8))
feature_importance_df.plot(kind="barh", ax=ax)
plt.gca().invert_yaxis()
plt.title("Feature Importance Comparison Across Models", fontsize=14, weight='bold')
plt.xlabel("Normalized Importance Score")
plt.grid(True)
plt.tight_layout()
plt.show()

# Print feature importance table
print(feature_importance_df.round(3))

# Re-Evaluate Tuned Model Performances

# Initialize new performance list
performance_list_tuned = []

# Evaluate all tuned models
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    performance_list_tuned.append({"Model": name, "RMSE": rmse, "R2": r2})

# Create performance DataFrame
performance_df_tuned = pd.DataFrame(performance_list_tuned)
performance_df_tuned.rename(columns=lambda x: x.strip().upper(), inplace=True)
performance_df_tuned = performance_df_tuned.round({"RMSE": 3, "R2": 3})
performance_df_tuned = performance_df_tuned.sort_values(by="R2", ascending=False).reset_index(drop=True)
performance_df_tuned.index += 1
performance_df_tuned.index.name = "Rank"

# Print tuned performance table
print("\nTuned Model Performance:")
print(performance_df_tuned)